In [ ]:
# SeleniumのActionChainsクラスを使用。
# 自分で調べる！

# データを取得するページは以下のURLから。
# https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month

# 1. HTMLから株価データを抽出する関数の作成。
# 関数名　→　extract_stock_data
#           ・HTMLを引数に取る。
#           ・BeautifulSoupライブラリを使用。
#           ・日付，始値，高値，安値，終値を取得し，それらをリストにする。
#           ・日付は「年-月-日」の表示で，0埋めをする。
#           ・上記で作成したリストを関数の戻り値とする。

# 2. Webページへアクセスしてデータを抽出する関数の作成。
# 関数名　→　get_stock_values
#           ・Selenium WebDriverオブジェクトとURLを引数に取る
#           ・Seleniumライブラリを使用する。
#           ・グラフの中央にマウスを移動させ，そこからグラフ幅の半分の値だけ
#           　右にマウスをずらすことで，グラフの右端にマウスを移動させる。
#           ・1pxずつ左にマウスをずらすことで，データを取得する。
#           ・データの抽出には，作成したextract_stock_data()関数を使用する。

# 3. mainコードの記述（get_stock_values()関数を使用して株価データを抽出）。
# スクレイピングの開始時間と終了時間を取得し，スクレイピングにかかった時間を計算して表示。
# また取得した日付，始値，高値，安値，終値をこの順で表示。

# 日付，始値，高値，安値，終値のCSSセレクタ
# #highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr > td

# グラフのセレクタ
# #highcharts-0 > svg > g.highcharts-series-group


In [3]:
# 必要なライブラリをインポート。
import time
import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

# ヘッドレスモードで起動するためのオプションを設定。
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 空のリストを作成。
stock_data_list = []

# Chromeを立ち上げる。
chrome_driver = webdriver.Chrome(options=chrome_options)

# 今回開くページのurl。
url = 'https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month'

# 日経平均株価のページにアクセス。
chrome_driver.get(url)

# ページのHTMLをBeautifulSoupで解析。
soup = BeautifulSoup(chrome_driver.page_source, 'html.parser')

# 下に300スクロール。
chrome_driver.execute_script("window.scrollTo(0, 300)")

# HTMLを引数に取り，そのHTMLを解析することで株価データを抽出する関数を作成。
def extract_stock_data(soup):
    # 取得したテーブルからtdタグの中身をリストで取得。
    for i in range(1, 6):
        # 更新されたページのHTMLをBeautifulSoupで解析。
        soup = BeautifulSoup(chrome_driver.page_source, 'html.parser')
        # 日付，始値，高値，安値，終値をこの順で取得。
        stock_price_data = soup.select_one(f'#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child({i})')
        # 取得した値を文字列に変換。
        stock_price_value = stock_price_data.string

        # 日付を指定の表示方法に変更
        if i == 1:
            date = datetime.datetime.strptime(stock_price_value, '%Y/%m/%d')
            stock_price_date = date.strftime('%Y-%m-%d')
            judgement = stock_price_date not in stock_data_list

            # リストが被らないときだけリストを更新。
            if judgement == True:
                stock_data_list.clear()
                stock_data_list.append(stock_price_date)
            else:
                break

        else:
            stock_data_list.append(stock_price_value)

    # もしかぶりがなければリストを戻り値として返す。
    if judgement == True:
        return stock_data_list

# グラフを取得。
target = chrome_driver.find_element(By.CSS_SELECTOR, '#highcharts-0 > svg > g.highcharts-series-group')

# グラフの横幅を取得。
target_width = target.rect['width']

# グラフの横幅を出力。
print(target_width)

# Selenium WebDriverオブジェクトとURLを引数に取り，Webページから株価データを抽出する関数を作成。
def get_stock_values(chrome_driver, url):
    # リストを初期化。
    stock_data_list.clear()

    # ActionChains(chrome_driver)を変数actionsに代入。
    actions = ActionChains(chrome_driver)

    # マウスカーソルをグラフの中央に移動。
    actions.move_to_element(target)

    # マウスカーソルをグラフの右端に移動。
    actions.move_by_offset((target_width // 2), 0)

    # 上記のアクションを連続で行う。
    actions.perform() 

    # 1秒待機。
    time.sleep(1)

    # マウスを左に移動させながらリストを取得。
    for j in range(1, target_width + 1):

        a = extract_stock_data(soup)

        if a == None:
            pass

        else:
            # リストを出力。
            print(a)
        
        # マウスカーソルを左に移動。
        actions.move_by_offset(-1,0)

        # 一連の動作を実行。
        actions.perform()

get_stock_values(chrome_driver, url)

# ブラウザを閉じる。
chrome_driver.quit()

559
['2024-08-07', '始値: 34,122.35', '高値: 35,849.77', '安値: 33,739.46', '終値: 35,089.62']
['2024-08-06', '始値: 32,077.33', '高値: 34,911.8', '安値: 32,077.33', '終値: 34,675.46']
['2024-08-05', '始値: 35,249.36', '高値: 35,301.18', '安値: 31,156.12', '終値: 31,458.42']
['2024-08-02', '始値: 37,444.17', '高値: 37,471.52', '安値: 35,880.15', '終値: 35,909.7']
['2024-08-01', '始値: 38,781.56', '高値: 38,781.56', '安値: 37,737.88', '終値: 38,126.33']
['2024-07-31', '始値: 38,140.77', '高値: 39,188.37', '安値: 37,954.38', '終値: 39,101.82']
['2024-07-30', '始値: 38,241.35', '高値: 38,531.16', '安値: 38,076.7', '終値: 38,525.95']
['2024-07-29', '始値: 38,139.12', '高値: 38,719.7', '安値: 38,010.01', '終値: 38,468.63']
['2024-07-26', '始値: 37,792.87', '高値: 38,105.96', '安値: 37,611.19', '終値: 37,667.41']
['2024-07-25', '始値: 38,524.2', '高値: 38,557.79', '安値: 37,825.58', '終値: 37,869.51']
['2024-07-24', '始値: 39,336.74', '高値: 39,643.27', '安値: 39,096.5', '終値: 39,154.85']
['2024-07-23', '始値: 39,864.09', '高値: 39,927.89', '安値: 39,519.91', '終値: 39,594.39']
['2024

In [2]:
chrome_driver.quit()